<center>
    <h1> INF393 - Maquinas de Aprendizaje  </h1>
    <h2> Tarea 3: Métodos No Lineales para Clasificación </h2>
    <h5> Alumnos: Ignacio Espinoza & Daniel Rivera</h5>
</center>

<h2> PARTE 3: Reconocimiento de Imágenes en CIFAR 10 </h2>

<p>La matriz de elementos consisten en una matriz de $10000 × 3072$ , en donde cada fila de esa matriz corresponde a una imagen en formato $RGB$, y para cada imagen, o sea cada fila, los primeros $1024$ valores vienen del canal $R$, los siguientes $1024$ del canal $G$, y los últimos $1024$ del canal $B$.
Para cada canal, las imágenes han sido vectorizadas por filas, de modo que los primeros 32 valores del canal
R corresponden a la primera fila de la imagen. Por otro lado, el elemento (labels) del diccionario contiene
una lista de 1000 valores enteros entre 0 y 9 que identifican las clases excluyentes  las cuales son 
<i>gato, perro, rana, caballo, pajaro, ciervo, avion, automovil, camion y barco:</i>.</p>

<h4> Generando Dataset </h4>

In [1]:
from scipy.misc import imread
import cPickle as pickle
import numpy as np
import os
import random

from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

def unpickle(file):
   
   fo = open(file, 'rb')
   dict = pickle.load(fo)
   fo.close()
   return dict

def load_CIFAR_one(filename):
   with open(filename, 'rb') as f:
      datadict = pickle.load(f)
      X = datadict['data']
      Y = datadict['labels']
      return X, np.array(Y)

def load_CIFAR10(PATH):
   xs = []
   ys = []
   for b in range(1,6):
      f = os.path.join(PATH, 'data_batch_%d' % (b, ))
      X, Y = load_CIFAR_one(f)
      xs.append(X)
      ys.append(Y)
   Xtr = np.concatenate(xs)
   Ytr = np.concatenate(ys)
   del X, Y
   Xte, Yte = load_CIFAR_one(os.path.join(PATH, 'test_batch'))
   
   i = random.randint(0, 40000)
   Xval = Xtr[i:(i+10000)]
   Yval = Ytr[i:(i+10000)]

   return Xtr, Ytr, Xte, Yte, Xval,Yval


Using Theano backend.


In [2]:
def scale_data1(Xtr,Xte,Xval):
   return Xtr/255.0 ,Xte/255.0 ,Xval/255.0

def scale_data2(Xtr,Xte):
   scaler = StandardScaler()
   Xtr_scaled = scaler.fit_transform(Xtr[:20000])
   Xte_scaled = scaler.fit_transform(Xte)
   #Xval = scaler.fit_transform(Xval)
   return Xtr_scaled, Xte_scaled

def scaler_function(Xtr,Xte,Xv,scale=True):
   scaler = StandardScaler(with_std = scale).fit(Xtr[:22000])
   Xtr_scaled = scaler.transform(Xtr[:22000])
   Xte_scaled = scaler.transform(Xte)
   Xv_scaled = scaler.transform(Xte)
   return Xtr_scaled, Xte_scaled , Xv_scaled

<h4> Escalando Datos </h4>

In [7]:
Xtr, Ytr, Xte, Yte, Xval, Yval = load_CIFAR10('.')

Xtr_scaled, Xte_scaled, Xval_scaled = scaler_function(Xtr,Xte,Xval)


In [8]:
#CONSIDERACION ESPECIAL POR PROBLEMAS DE MEMORIA, SE REDUCE EL DATASET
Xtr,Xte, Xval = scaler_function(Xtr,Xte,Xval)
Ytr = to_categorical(Ytr[:22000])
Yte = to_categorical(Yte)
Yval = to_categorical(Yval)

<h3> Implementación : Redes Neuronales </h3>

<p> Al implementar redes neuronales en un problema de clasificación múltiple, debemos generar una función de hipótesis de modo que lo que retorne sea un vector de valores, el cual será un vector de $0$ con excepción de la clase que será clasificada, asignandole un valor igual a $1$.</p>


$$y^{(i)} = \begin{bmatrix}0 \\ 0 \\ 1 \end{bmatrix} , \begin{bmatrix}0 \\ 1 \\ 0 \end{bmatrix}, \begin{bmatrix}1 \\ 0 \\ 0 \end{bmatrix} $$

<img src="NeuralNetwork3.png" />

<h4> ¿Cual es el mejor criterio para seleccionar cantidad de capas ocultas y neuronas por capa ? </h4>

<p> No existe una regla de oro para la selección de capas ocultas y nodos , por lo que es necesario recurrir a heurísticas y pruebas de entrenamiento para encontrar la mejor configuración. De todos modos, existen ciertas convenciones, acorde a la estructua del dataset. A continuación se presentarán las heurísticas importantes:</p>

<h4> Heurística #1 </h4>
<h3>$$ N_h = \frac{N_s}{(\alpha * (N_i + N_o))}$$</h3>

<ul>
<li> $N_i$ = número de neuronas de entrada</li>
<li> $N_o$ = número de neuronas de salida</li>
<li> $N_s$ = número de elementos en el dataset</li>
<li> $ \alpha$ = valor arbitrario entre 5-10</li>
</ul>

<p> Con esta heurística, se obtiene un valor de 3 nodos en una capa oculta, por lo que a continuación entrenará una red neuronal con 1 capa oculta y 3 nodos </p>

<h4>Heurística #2:</h4>
<p>"1 Capa Oculta es Suficiente"</p>
<p>Elegir un valor promedio entre número neuronas de entrada y salida</p>

In [9]:
def neural_network_classifier(Xtr, Ytr, Xte, Yte, Xv, Yv):

    model = Sequential()
    model.add(Dense(50, input_dim = Xtr.shape[1], init = 'uniform', activation = 'sigmoid'))
    model.add( Dropout(0.1) )
    model.add(Dense(10, init = 'uniform', activation = 'softmax'))
    model.compile(optimizer = SGD( lr = 0.05), loss = 'binary_crossentropy', metrics = ['accuracy'] )
    model.fit(Xtr, Ytr, nb_epoch = 50, batch_size = 32, verbose = 1, validation_data = (Xte,Yte))
    scores = model.evaluate(Xte, Yte)
    test_acc = scores[1]
    scores = model.evaluate(Xv, Yv)
    val_acc = scores[1]
    scores = model.evaluate(Xtr, Ytr)
    train_acc = scores[1]
    
    print "Test Accuracy: " + str(test_acc)
    

    print "Train Accuracy: " + str(train_acc)
    
    
    print "Evaluate Accuracy: " + str(val_acc)

#neural_network_classifier(Xtr, Ytr, Xte, Yte, Xval, Yval)


<h3> RELU 55 NODOS</h3>

<table>
<tr>
<th>Experimento </th>
<th> Training </th>
<th>Test</th>
<th>Eval </th>
</tr>

<tr>
<th>SIGMOID 2 CAPAS  50 NODOS</th>
<td>0.900209069772</td>
<td>0.90017997818</td>
<td>0.900379990196</td>
</tr>

<tr>
<th>SIGMOID 100 NODOS</th>
<td> 0.911720000458</td>
<td>0.919250000217</td>
<td> 0.895939997864</td>
</tr>

<tr>
<th>RELU 100 NODOS s</th>
<td>0.960490908059</td>
<td>0.909230000687</td>
<td>0.858430000114</td>
</tr>

<tr>
<th>SIGMOID 55 NODOS</th>
<td>0.910940000439</td>
<td>0.916822728157</td>
<td> 0.894919999218</td>
</tr>

<tr>
<th>Sigmoid 2 NO DROPOUT</th>
<td>0.900081810518</td>
<td>899899991989</td>
<td>0.83709999885</td>
</tr>

<tr>
<th>1 CAPAS , 3 NODOS</th>
<td>0.900099984776</td>
<td>0.89994998455</td>
<td>0.830809999466</td>
</tr>

<tr>
<th>1 CAPAS, 100 NODOS</th>
<td>0.914449999289</td>
<td>0.907350000858</td>
<td> 0.9144499992896</td>
</tr>



</table>

Como se observa, el mejor resultado se obtuvo con 55 nodos, pero tambien usando 2 capas y 50 nodos se obtiene un accurace del 90%. Este resultado no determina ninguna correlación entre el número de nodos y el número de capas ocultas y la distribución existente entre ellas.

Por otro lado, la función sigmoid, en comparación con Relu, mostro resultados peores, esto refleja la estructura dispersa de los datos de entrenamiento

<h3>Extract Features</h3>

In [11]:
from top_level_features import hog_features
from top_level_features import color_histogram_hsv
from top_level_features import extract_features

In [13]:
#Xtr, Ytr, Xte, Yte, Xval, Yval = load_CIFAR10(".")

train_features = extract_features(Xtr,[color_histogram_hsv]) #extrae histogramas de color
test_features = extract_features(Xte,[color_histogram_hsv]) #extrae histogramas de color
val_features = extract_features(Xv,[color_histogram_hsv])
                                
train_features , test_features, val_features = scaler_function(train_feature,test_features,val_features)


#neural_network_classifier(train_features, Ytr, test_features, Yte, val_features, Yval)
#ERROR DE MEMORIA

MemoryError: 

<h3> Implementación SVM No Lineal </h3>
Es importante considerar que con SVM No Lineal, la  implementación de <i>sklearn</i> funciona correctamente con datasets superior a los 10000 elementos, por lo que se debe reducir la dimensionalidad del set de entrenamiento. Esto puede conllevar a un sobreajusta dada la poca cantidad de datos relativo al resto de los datos.

In [ ]:
def NoLineal_SVM_classifier(Xtr, Ytr, Xte, Yte, Xv, Yv):
    print ("hola")
    clf_kernel_rbf = SVC(C = 500, kernel = 'rbf')
    clf_kernel_rbf.fit(Xtr, Ytr)

    #clf_kernel_poly = SVC(C = 10, kernel = 'poly', degree = 2, coef0 = 1)
    #clf_kernel_poly.fit(Xtr, Ytr)
    
    #training_kernel_score = clf_kernel_rbf.score(Xtr,Ytr)
    
    testing_kernel_score = clf_kernel_rbf.score(Xte,Yte)
    
    eval_kernel_score = clf_kernel_rbf.score(Xv,Yv)
    
    #training_poly_score = clf_kernel_poly.score(Xtr,Ytr)
    
    #testing_poly_score = clf_kernel_poly.score(Xte,Yte)
    
    #eval_poly_score = clf_kernel_poly.score(Xv,Yv)
    
    #print "Kernel Training Accuracy: " + str(training_kernel_score)
    
    print "Kernel Test Accuracy: " + str(testing_kernel_score)
    
    print "Kernel Eval Accuracy: " + str(eval_kernel_score)
    
    #print "Polynomial Training Accuracy: " + str(training_poly_score)
    
    #print "Polynomial Training Accuracy: " + str(testing_poly_score)
    
    #print "Polynomial Training Accuracy: " + str(eval_poly_score)
    

NoLineal_SVM_classifier(Xtr[:100], Ytr[:100], Xte[:100], Yte[:100], Xval[:100], Yval[:100])
    

<h4> Kernel Training Accuracy: 1.0 </h4>
<h4> Kernel Test Accuracy: 0.1  </h4>
<h4>Kernel Eval Accuracy: 0.097</h4>

Implementar un SVM No Lineal a un dataset de alta dimensionalidad conlleva a un gasto de recursos y tiempo elevado, a su vez, los resultados no son alentadores, obtiendo en general un accuracy cercano al 10%.Estos datos se obtuvieron con un coeficiente de regularización alto, o sea mas ajustado al set de entrenamiento, por lo que se observa overfitting. El mejor resultado se obtiene con un C cercano a 10 (menos sensible a ruido) con un accuracy en evaluación cercano al 40%. Se concluye que los SVM no lineales no logran generar un model eficiente para el problema de reconocimiento de imágenes.